<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/ISOTIPO-Color.jpg" title="Title text" width="20%" />

<hr style="height:2px;border:none"/>
<H1 align='center'> SVD Y PCA </H1>

<H3> INF-285 Computación Científica </H3>
<H3> Autor: Francisco Andrades</H3>

Lenguaje: Python

Temas:

    - SVD
    - PCA
    - Image Compression
<hr style="height:2px;border:none"/>

# Introducción

La compresión de Imágenes utilizando *SVD* se basa en que  la matriz $\Sigma$ representa los valores singulares de la matriz original, entonces se puede obtener una aproximación de la imagen original minimizando el rango de la matriz al eliminar los  valores singulares de menor valor, ya que estos representan una "menor información" de la imagen. De este forma, por ejemplo si $\Sigma$ es de tamaño $n\times n$, se pueden omitir los $\sigma$ menos significativos obteniendo $\tilde{\Sigma}$ de tamaño $m\times m$, $m<n$.

Por otro lado, también se puede utilizar el análisis de componentes principales (PCA) para la compresión de imágenes al reducir la dimensión de la matriz de la imagen y proyectar esas nuevas dimensiones en una nueva imagen reteniendo la información importante de la imagen original

En esta tarea se busca comprimir un archivo *GIF*, el cual consiste de una secuencia de multiples imagenes, utilizando *SVD* y *PCA* para poder comparar ambos métodos y analizar la relación entre ambos.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageSequence
import matplotlib.image as mpimg

In [2]:
def plotAnimation(animation):
    """
    Parameters
    ----------
    animimation         : (frames, rows, cols) array
                          GIF array

    Returns
    -------
    Animation plots      : None
    """
    i = 0
    for frame in animation:
        plt.imshow(frame, cmap=plt.cm.gray)
        plt.axis('off')
        plt.show()
        i = i+1

In [3]:
def gifToArray(gif_file):
    """
    Parameters
    ----------
    gif_file             : string
                          GIF path

    Returns
    -------
    data                 : (frames, rows, cols) array
                          NumPy array with GIF pixel values
    """
    im = Image.open(gif_file)
    data = list()
    for frame in ImageSequence.Iterator(im):
        tmp = np.array(im.convert('L'))
        data.append(tmp)
    data = np.array(data)
    return data



Podemos considerar un *GIF* como una colección de $p$ *frames*, donde un *frame* es una martriz $F\in\mathbb{R}^{r\times c}$ con $r$ el número de filas y $c$ en número de columnas de esta imagen. Ahora, si $(f_k)_{i,j}$ corresponde al elemento $i,j$ del $k$-ésimo *frame*, vamos a definir $\mathbf{f}_{i,j}=\langle (f_1)_{i,j}, (f_2)_{i,j},\dots,(f_p)_{i,j}\rangle$,
es decir, este vector corresponde a los valores de los $p$ frames de la coordenada $(i,j)$ del *GIF*.

Finalmente, para trabajar con los algoritmos, vamos a construir la matriz $G \in \mathbb{R}^{q\times p}$, donde $q=r\times c$ de cada *frame*, y que se define como:

\begin{equation}
    G = 
    \left[
    \begin{array}{c}
        \mathbf{f}_{1,1} \\ \hline
        \mathbf{f}_{1,2} \\ \hline
        \dots \\ \hline
        \mathbf{f}_{r,c}
    \end{array}
    \right]
\end{equation}

----
## Funciones a Implementar

1. Crear la función ```createG(data)``` que recibe ```data``` el arreglo ```NumPy``` con la información del GIF, y retorna el arreglo $G$ definido anteriormente. (10 puntos)

In [4]:
def createG(data):
    """
    Parameters
    ----------
    data             : (frames, rows, cols) array
                       NumPy array with GIF pixel values

    G                : (q, p) array
                       G matrix
    """
    G = list()
    for frame in data:
        fila_nueva = []
        for row in frame:
            for i in row:
                fila_nueva.append(i)
        G.append(np.array(fila_nueva))
    G = np.array(G)
    return G.transpose()

2. Crear la función ```restoreGIF(data)``` que recibe los datos procesados ```data``` y ```shape``` que contiene la tupla ```(frames, rows, cols)```, la dimensión original del *GIF*. Esta función retorna la reconstrucción del GIF. (10 puntos)

In [5]:
def restoreGIF(data, shape):
    """
    Parameters
    ----------
    data             : (q, p) array
                       G matrix
    shape            : tuple (frames, rows, cols) 
    Returns
    -------
    reshaped_data    : (frames, rows, cols) array
                       NumPy array with GIF pixel values
                       
    """
    
    n_frames = shape[0]
    n_rows = shape[1]
    n_col = shape[2]
    reshaped_data = []
    G = data.transpose()
    
    for frame in G:
        frame_original = [] 
        for cont in range(n_rows):
            row_original = frame[cont*n_col:(cont+1)*n_col]
            frame_original.append(row_original)
        

        reshaped_data.append(frame_original)
    
    reshaped_data = np.array(reshaped_data)
    return reshaped_data

### SVD
3. Implementar la función ```G_SVD(G, m)``` que reciba la matriz $G$ y los $m$ componentes que se utilizarán para comprimir el *GIF* utilizando *SVD*. La función debe retornar $U$, $\textrm{diag}(\Sigma)$ y $V^T$. Además, implementar la función ```SVD_G(U, s, Vt)``` que recibe las matrices generadas por el *SVD* y retorne la reconstrucción de la matriz $G$. (30 puntos)

In [6]:
# G to SVD
def G_SVD(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    U             : (q, m)-array
                    SVD U matrix
    s             : m-array
                    Singular values
    Vt            : (m, p)-array
                    SVD V^T matrix 
    """
    # Apply SVD
    u, n, vt =np.linalg.svd(G,full_matrices = False)
    
    #tomar las primeras m columnas de u
    
    U = []
    for row in u:
        fila_nueva = row[:m]
        U.append(fila_nueva)
        
    
    #tomar las primeras m filas y columnas de n
    
    s = n[:m]
    #tomar las primeras m filas de vt
    Vt = vt[:m]
    
    U = np.array(U)
    s = np.array(s)
    Vt = np.array(Vt)
    return U, s, Vt

# SVD to 'compressed' G
def SVD_G(U, s, Vt):
    """
    Parameters
    ----------
    U             : (q, m)-array
                    SVD U matrix
    s             : m-array
                    Singular values
    Vt            : (m, q)-array
                    SVD V^T matrix 
    Returns
    -------
    B             : (p, q)-array
                    "Compressed" G
    """
    
    b1 = []
    cont = 0
    for row in Vt:
        b1.append(row*s[cont])
        cont = cont+1
        
    B = U.dot(b1)
    B = np.array(B)
    return B

### PCA
4. Implementar la función ```G_PCA(G, m)``` que reciba la matriz $G$ y los $m$ componentes que se utilizarán para comprimir el *GIF* utilizando *PCA*. La función debe retornar $PC$, $Y$ y $\mu$. Además, implementar la función ```PCA_G(PC, Y, mu)``` que recibe las matrices generadas por *PCA* y retorne la reconstrucción de la matriz $G$. Para esto debe utilizar la funcion de SVD implementada en el punto anterior. (35 puntos)

In [7]:
def G_PCA(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    PC             : (p, m)-array
                     first m principal components
    Y             : (q,m)-array
                    PC Scores 
    mu           : (p)-array
                    Average per column 
    """

    mu = G.mean(0)
    
    Z = G - mu
    
    U,s,Vt = G_SVD(Z,m)
    
    v = Vt.transpose()
    
    PC = []
    for row in v:
        fila_nueva = row[:m]
        PC.append(fila_nueva)
        
    PC = np.array(PC) 
    
    aux = np.zeros((m, m))
    np.fill_diagonal(aux, s)
        
    Y = U.dot(aux)
    
    return  PC, Y, mu

In [8]:
def PCA_G(PC, Y, mu):
    """
    Parameters
    ----------
    PC             : (p, m)-array
                     first m principal components
    Y             : (q,m)-array
                    PC Scores 
    mu           : (p)-array
                    Average per column 
    Returns
    -------
    B            : (q, p)-array
                    "Compressed" G
    """
    
    Vt = PC.transpose()
    
    matriz = Y.dot(Vt)
    
    B = matriz + mu
    return B

## Preguntas

Para responder las siguientes preguntas, debe implementar las funciones propuestas

#### 1. ¿Cuál sería el costo de almacenamiento en MB usando $m$ vectores singulares? (5 puntos)

In [9]:
def SVD_size(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    size          : Float
                    total size of SVD return
    """
    U,s,Vt = G_SVD(G,m)
    size1 = U.nbytes
    size2 = Vt.nbytes
    size3 = s.nbytes
    size = size1 + size2 + size3
    size = size/1024
    return size/1024

def PCA_size(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    size          : Float
                    total size of PCA return
                    
    """
    PC , Y , mu = G_PCA(G, m)
    size1 = PC.nbytes
    size2 = Y.nbytes
    size3 = mu.nbytes
    size = size1 + size2 + size3
    size = size/1024
    return size/1024

#### 2. ¿Cuál sería el *gif* resultante con $m$ componentes? (5 puntos)

In [10]:
def print_animation_SVD(G, m,shape):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    La funcion no debe retornar nada, solo mostrar las imagenes de los frames reconstruidos
    """
    U,s,Vt = G_SVD(G,m)

    B = SVD_G(U,s,Vt)
    restore1 = restoreGIF(B,shape)
    plotAnimation(restore1)
    return

def print_animation_PCA(G, m,shape):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    La funcion no debe retornar nada, solo mostrar las imagenes de los frames reconstruidos
    """
    
    PC , Y , mu = G_PCA(G, m)
    B = PCA_G(PC, Y, mu)
    restore1 = restoreGIF(B,shape)
    plotAnimation(restore1)
    return

#### 3. ¿Cual sería el error en función de $m$? (Calcule el error utilizando la norma-2) (5 puntos)

Considere calcular el error de la siguiente manera: $||G-B_m||_2$, donde $G$ corresponde a la matriz definida anteriormente y $B_m$ a la matriz "comprimida" utilizando los métodos correspondientes para un $m$ particular.

In [11]:
def compression_error_SVD(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    error          : Float
                    total size of PCA return
    """
    
    U,s,Vt = G_SVD(G,m)

    B = SVD_G(U,s,Vt)
    
    H = G-B
    error = np.linalg.norm(H,2)
    return error

def compression_error_PCA(G, m):
    """
    Parameters
    ----------
    G             : (q, p)-array
                    G matrix
    m             : int
                    Number of components
    Returns
    -------
    error         : Float
                    total size of PCA return
    """
    PC , Y , mu = G_PCA(G, m)
    B = PCA_G(PC, Y, mu) 
    H = G-B
    error = np.linalg.norm(H,2)
    return error

# Prueba

Para verificar sus algoritmos, pruebe las funciones desarrolladas para $m=10$.

In [14]:
data = gifToArray("somebody.gif") 
shape =data.shape
G = createG(data)
print("Size SVD:",SVD_size(G,10))
print("Size PCA:",PCA_size(G,10))
#print_animation_SVD(G, 10,shape) #descomentar para imprimir los frames
#print_animation_PCA(G, 10,shape) #descomentar para imprimir los frames
print("Error SVD:",compression_error_SVD(G, 10))
print("Error PCA:",compression_error_PCA(G, 10))

Size SVD: 6.3668060302734375
Size PCA: 6.368255615234375
Error SVD: 10888.513291792182
Error PCA: 10872.976738591024
